### General analysis of november data

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
from retrieve_data import read_waze_data
from utils import constants
import os
import datetime as dt
import numpy as np

In [3]:
[df_tt, df_r] = read_waze_data('21.11.2018')

In [4]:
dict_dir = os.path.join(constants.waze_travel_times_dir, 'Dict/dicc-tramos-waze.xlsx')
stretch_dict = pd.read_excel(dict_dir)

In [5]:
#stretch_dict.head()

In [6]:
#df_tt.head()

In [7]:
def process_waze_data(df_tt, df_r):

    df_tt['date'] = df_tt['updatetime'].dt.date
    df_tt = df_tt.merge(df_r[['name', 'length']], on='name', how='left')
    df_tt = df_tt.loc[df_tt['length'].isnull() == False, :]
    df_tt['time/length[s/km]'] = (df_tt['time'] / (df_tt['length']))*1000
    df_tt['length/time[km/h]'] = (df_tt['length'] / (df_tt['time']))*3.6

    return df_tt

In [8]:
df_tt = process_waze_data(df_tt, df_r)

In [9]:
df_tt.head()

,name,time,historictime,jamlevel,updatetime,jams,alerts,date,length,time/length[s/km],length/time[km/h]
0,TramoCDDNS1,146,137,2,2018-10-23 14:50:45,{},{},2018-10-23,501,291.417166,12.353425
1,TramoCDDSN3,154,163,2,2018-10-23 14:50:45,{},{},2018-10-23,456,337.719298,10.659740
2,TramoIraPO1,77,88,1,2018-10-23 14:50:45,{},{},2018-10-23,326,236.196319,15.241558
3,TramoTresAntonioSN1,50,74,0,2018-10-23 14:50:45,{},{},2018-10-23,498,100.401606,35.856000
4,TramoTresAntoniosSN2,161,173,0,2018-10-23 14:50:45,{},{},2018-10-23,668,241.017964,14.936646


Select the dates

In [10]:
start_date = dt.date(2018, 11, 5)
except_7_nov = dt.date(2018,11,7) #energy cut-off
except_12_nov = dt.date(2018,11,12) #hail storm
end_date = dt.date(2018, 11, 18)

In [11]:
mask = (df_tt['date'] >= start_date) & (df_tt['date'] <= end_date) & (df_tt['date'] != except_7_nov) & (df_tt['date'] != except_12_nov)
nov_df = df_tt.loc[mask]
print(len(nov_df.index))

200604


In [12]:
include_24_oct = dt.date(2018,10,24)
include_19_nov = dt.date(2018,11,19)

In [13]:
include_mask = (df_tt['date'] == include_24_oct) | (df_tt['date'] == include_19_nov)
replacing_df = df_tt.loc[include_mask]
print(len(replacing_df.index))

31392


In [14]:
analyze_df = pd.concat([nov_df,replacing_df])
print(len(analyze_df.index))

231996


In [15]:
def day_type(x):
    if (0<=x.weekday()<5):
        return 'L'
    elif (x.weekday()==5):
        return 'S'
    elif (x.weekday()==6):
        return 'D'

In [16]:
analyze_df['day'] = analyze_df['date'].apply(day_type)
analyze_df.sort_values(by=['updatetime'], inplace = True)
analyze_df['updatetime_tmp'] = analyze_df['updatetime'].apply(lambda x: x.replace(month=1,day=1)) #ugly

In [17]:
#analyze_df.head()

In [18]:
grouped_df_tt = analyze_df.groupby(['name', 'length', 'day', pd.Grouper(freq='15min', key='updatetime_tmp')])['time/length[s/km]','length/time[km/h]'].mean()
grouped_df_tt.reset_index(inplace=True)
grouped_df_tt['time'] = grouped_df_tt['updatetime_tmp'].dt.time

A simple test

In [19]:
type_of_days = 3
unique_sections = len(grouped_df_tt['name'].unique())
n_time_windows = 24*60/15

In [20]:
len(grouped_df_tt.index) == type_of_days * unique_sections * n_time_windows

True

Some other things

In [21]:
grouped_df_tt.drop(columns = 'updatetime_tmp', inplace=True)

In [22]:
grouped_df_tt['end_time'] = grouped_df_tt['time'].shift(-1)

In [23]:
#grouped_df_tt.tail()

In [24]:
general_file = os.path.join(constants.travel_times_dir,'analysis/mean_travel_times.csv')

In [25]:
grouped_df_tt.to_csv(general_file, sep=';')

Some personalization

In [26]:
routes_excluded = stretch_dict.loc[(stretch_dict['project'] == 'reversibilidad andrés bello')
                                   | (stretch_dict['project'] == 'test traffic'),'name'].unique().tolist()

In [27]:
grouped_df_tt = grouped_df_tt[~grouped_df_tt['name'].isin(routes_excluded)]
grouped_df_tt = grouped_df_tt.merge(stretch_dict[['name','main street','sense','eje_csch','tramo_csch']], on='name', how='left')
grouped_df_tt = grouped_df_tt.loc[(grouped_df_tt['time']>=dt.time(7,0,0))&(grouped_df_tt['time']<=dt.time(21,45,0)),:]
grouped_df_tt.drop(columns = 'length/time[km/h]', inplace=True)

Get the only-car stretch and adding some information

In [28]:
only_car_stretchs = grouped_df_tt.loc[grouped_df_tt['eje_csch'].isnull(),'name'].unique()

In [29]:
def changing_line(x):
    x = x.replace("[","")
    x = x.replace("]","")
    x = x.replace("(","")
    x = x.replace(")","")
    return pd.Series(x.split(","))

In [30]:
only_car = []
for stretch in only_car_stretchs:
    line = df_r.loc[df_r['name']==stretch, 'line'].apply(changing_line)

    line.columns = [np.arange(len(line.columns)) % 2, np.arange(len(line.columns)) // 2]
    line = line.stack().reset_index(drop=True)
    line.columns = ['lon','lat']

    lat = line.loc[:,'lat'].tolist()
    lon = line.loc[:,'lon'].tolist()
    
    only_car.append({'Nombre Big Data': stretch, 'lat_inicio': lat[0], 'lon_inicio': lon[0], 'lat_fin': lat[-1], 'lon_fin': lon[-1]})
    
only_car_df = pd.DataFrame(only_car)

In [31]:
columns_names = {'name': 'Nombre Big Data', 'length': 'Largo [m]', 'day': 'Día', 
           'time/length[s/km]': 'Tiempo Medio [s/km]', 'time':'Inicio', 'end_time': 'Fin', 'main street': 'Eje',
           'sense': 'Sentido', 'eje_csch': 'Nombre Camila', 'tramo_csch': 'Tramo GPS Camila'}

grouped_df_tt.rename(columns = columns_names, inplace=True)

In [32]:
jgibson_tt_path = os.path.join(constants.travel_times_dir,'analysis/tiempos_medios.xlsx')
jgibson_only_car_path = os.path.join(constants.travel_times_dir,'analysis/tramos_solo_autos.xlsx')

In [33]:
grouped_df_tt.to_excel(jgibson_tt_path, index=False)
only_car_df.to_excel(jgibson_only_car_path, index=False)